# North Atlantic and Caribbean Basins

In [1]:
import xarray as xr
from matplotlib.pylab import *
import numpy as np
import pandas as pd
from geopy import distance
from mpl_toolkits.axes_grid1 import make_axes_locatable
from geographiclib.geodesic import Geodesic
import time


In [2]:
def get_bearing(lat1, lat2, long1, long2):
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    return brng

In [3]:
xds_tracks = xr.open_dataset('Allstorms.ibtracs_wmo.v03r09.nc')

In [4]:
xds_tracks.basin.key

'0 = NA - North Atlantic\n1 = SA - South Atlantic\n2 = WP - West Pacific\n3 = EP - East Pacific\n4 = SP - South Pacific\n5 = NI - North Indian\n6 = SI - South Indian\n7 = AS - Arabian Sea\n8 = BB - Bay of Bengal\n9 = EA - Eastern Australia\n10 = WA - Western Australia\n11 = CP - Central Pacific\n12 = CS - Carribbean Sea\n13 = GM - Gulf of Mexico\n14 = MM - Missing'

In [25]:
xds_tracks = xds_tracks.sel(storm = np.unique(np.where((xds_tracks.basin == 0) | (xds_tracks.basin == 12))[0]))

In [26]:
xds_tracks

<xarray.Dataset>
Dimensions:          (center: 26, storm: 1816, time: 137)
Dimensions without coordinates: center, storm, time
Data variables:
    storm_sn         (storm) |S13 ...
    name             (storm) |S57 ...
    numObs           (storm) int16 ...
    season           (storm) int16 ...
    track_type       (storm) int8 ...
    genesis_basin    (storm) int8 ...
    num_basins       (storm) int8 ...
    basin            (storm, time) float32 0.0 0.0 0.0 0.0 ... nan nan nan nan
    wind_avg_period  (center) float32 ...
    source           (center) |S10 ...
    time_wmo         (storm, time) datetime64[ns] ...
    lat_wmo          (storm, time) float32 ...
    lon_wmo          (storm, time) float32 ...
    alt              (storm, time) float32 ...
    wind_wmo         (storm, time) float32 ...
    pres_wmo         (storm, time) float32 ...
    sub_basin        (storm, time) float32 ...
    nature_wmo       (storm, time) float32 ...
    source_wmo       (storm, time) float32 ...
    dist2land        (storm, time) float32 ...
    landfall         (storm, time) float32 ...
Attributes:
    Title:                IBTrACS-WMO: NetCDF reformat
    Version:              v03r09
    Description:          IBTrACS-WMO data reformatted to contain \nall data ...
    cdm_datatype:         Trajectory
    trajectoryDimension:  storm
    Conventions:          CF-1.0
    metadata_link:        gov.noaa.ncdc:C00834

In [27]:
import datetime

d = datetime.date(1997, 12, 31)
np.datetime64(d)

numpy.datetime64('1997-12-31')

In [28]:
lats = []
lons = []
pres = []
time = []
st_idx = []
for i in xds_tracks.storm.values:
    idx = np.argwhere(xds_tracks.sel(storm = i).time_wmo[~np.isnan(xds_tracks.sel(storm = i).time_wmo)].values > np.datetime64(d))
   #idx me selecciona los datos para fechas a partir de 1998
    if len(idx) == 0: #esta condicion significa que es un array vacio, con lo cual no hay fechas a partir de 1998
        continue
    else:
        
        lats.append(xds_tracks.sel(storm = i).lat_wmo.values)
        lons.append(xds_tracks.sel(storm = i).lon_wmo.values)
        pres.append(xds_tracks.sel(storm = i).pres_wmo.values)
        time.append(xds_tracks.sel(storm = i).time_wmo.values)
        st_idx.append(i)

In [29]:
NA_basin = xr.Dataset({'latitud': (('storm','time'), np.array(lats)),
           'longitud':(('storm','time'), np.array(lons)),
           'presion': (('storm','time'), np.array(pres)),
           'fechas': (('storm','time'), np.array(time))},
          {'storm': np.array(st_idx),
          'time': xds_tracks.time.values})

In [30]:
NA_basin

<xarray.Dataset>
Dimensions:   (storm: 300, time: 137)
Coordinates:
  * storm     (storm) int64 1516 1517 1518 1519 1520 ... 1812 1813 1814 1815
  * time      (time) int64 0 1 2 3 4 5 6 7 8 ... 129 130 131 132 133 134 135 136
Data variables:
    latitud   (storm, time) float32 11.3 11.7 12.2 12.599999 ... nan nan nan nan
    longitud  (storm, time) float32 -25.4 -27.199999 -29.199999 ... nan nan nan
    presion   (storm, time) float32 1009.0 1009.0 1009.0 1008.0 ... nan nan nan
    fechas    (storm, time) datetime64[ns] 1998-07-27T12:00:00.000043520 ... NaT

In [31]:
NA_basin.to_netcdf('basinNA.nc4')